In [96]:
# Load libraries
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Set up and initialize the webdriver 
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(service=Service(), options=options)

# Define columns for data table
my_colums = ['Corregimientos', 'Mulino', 'Lombana', 'Martin', 'Romulo',
             'Zulay', 'Gaby', 'Maribel', 'Meliton', 'Electores Totales',
             'Electores Escrutados', 'Mesas Escrutadas', 'Votos Emitidos']

# Define the number of districts per province
provinces = [
    ('01', 'Bocas del Toro', 4), 
    ('02', 'Coclé', 6), 
    ('03', 'Colón', 6),
    ('04', 'Chiriquí', 14), 
    ('05', 'Darién', 3), 
    ('06', 'Herrera', 7),	
    ('07', 'Los Santos', 7), 
    ('08', 'Panamá', 6), 
    ('09', 'Veraguas', 12),
    ('10', 'Com. Guna Yala', 0), 
    ('11', 'Com. Emberá', 2), 
    ('12', 'Com. Ngäbe-Buglé', 9), 
    ('13', 'Panamá Oeste', 5),
    ('14', 'Com. Kuna de Madugandí', 0), 
    ('15', 'Com. Kuna Wargandí', 0),
    ('16', 'Com. Naso Tjër Di', 0),
]

In [100]:
def get_district_data(driver, columns=my_colums):    
    table_data = []
    driver.implicitly_wait(15)
    table_rows = driver.find_elements(By.TAG_NAME, "tr")
    for i, row in enumerate(table_rows):
        if i > 0:
            table_data.append(row.text.split("\n"))
    df = pd.DataFrame(columns=columns, data=table_data)
    if len(df) == 0:
        print("Error: No data found in table")
        return None
    else:
        return df

def get_district_name(driver):
    driver.implicitly_wait(15)
    try:
        title_element = driver.find_element(By.CLASS_NAME, "te-titulo-seccion")
        section_title = title_element.text.split(">")
        district_name = section_title[1].split(":")[1]
        district_name = district_name.strip().title()
        return district_name
    except IndexError:
        print("Error: No elements found with class name 'te-titulo-seccion'")
        return None

In [102]:
# Defina base URL
base_url = "https://resultados.te.gob.pa/resultados/100/presidente/1"

# Loop through all provinces
for p in range(len(provinces)):
    prov_record = provinces[p]
    prov_code = prov_record[0]
    prov_name = prov_record[1]
    n_districts = prov_record[2]
    print(prov_name.upper())
    
    # Loop through districts (if necessary)
    if n_districts > 0:
        districts_list = []
        for n in range(n_districts):
            extra_url = f"/provincia/{str(p+1)}/distrito/{str(n)}"
            driver.get(base_url + extra_url)
            d_name = get_district_name(driver)
            print(f"{dist_code} - {d_name}")
            df = get_district_data(driver)
            dist_code = "{:02d}".format(n+1)
            df['Prov_ID'] = prov_code
            df['Dist_ID'] = dist_code
            df['Provincia'] = prov_name
            df['Distrito'] = d_name
            districts_list.append(df)
    else:
        extra_url = f"/provincia/{str(p+1)}"
        driver.get(base_url + extra_url)
        df = get_district_data(driver)
        df['Prov_ID'] = prov_code
        df['Dist_ID'] = "NA"
        df['Provincia'] = prov_name
        df['Distrito'] = "NA"
        districts_list = [df]

    # Save to a CSV file
    df_prov = pd.concat(districts_list)
    file_name = f"data/presidenciales_{prov_code}.csv"
    df_prov.to_csv(file_name, index=False)

BOCAS DEL TORO
01 - Bocas Del Toro


TypeError: 'NoneType' object does not support item assignment